In [1]:
%pip install --use-feature=2020-resolver auto-sklearn

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [1]:
import sklearn.datasets
import sklearn.model_selection
import autosklearn.classification
import autosklearn.metrics
import timeit
import pandas as pd

In [2]:
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  return (X_train, X_test, y_train, y_test)

In [3]:
X_train, X_test, y_train, y_test = dataSetup("australian")

In [4]:
X_train

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14
647,0,250,67,0,9,0,86,0,0,0,1,0,0,1
163,0,135,91,1,4,4,52,1,1,7,0,0,66,1
380,0,91,123,0,10,0,105,1,1,10,0,0,48,150
603,1,329,83,0,5,2,11,1,1,3,1,0,3,110
432,1,269,176,0,5,2,31,0,1,11,0,0,13,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1,64,26,0,8,2,15,0,1,5,1,0,3,43
106,0,93,38,0,7,2,45,1,1,6,1,0,9,56
270,1,20,132,0,4,2,56,0,0,0,1,0,16,120
435,0,159,24,0,1,0,43,1,1,10,0,0,23,169


In [5]:
start = timeit.default_timer()
estimator_askl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=3600,seed=42,resampling_strategy='cv',metric=autosklearn.metrics.roc_auc,)
estimator_askl.fit(X_train, y_train, dataset_name='australian')
stop = timeit.default_timer()
print(stop - start)
prediction = estimator_askl.predict(X_test)

3595.6284023225307


In [6]:
(prediction == y_test).sum()/len(prediction)

0.861271676300578

In [7]:
print(estimator_askl.sprint_statistics())

auto-sklearn results:
  Dataset name: australian
  Metric: roc_auc
  Best validation score: 0.925316
  Number of target algorithm runs: 405
  Number of successful target algorithm runs: 392
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0



In [8]:
from pprint import pprint
pprint(estimator_askl.show_models(), indent=4)

{   31: {   'cost': 0.07881950300781468,
            'ensemble_weight': 0.08,
            'estimators': [   {   'balancing': Balancing(random_state=42),
                                  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x14661e882280>,
                                  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x14661f9b9b20>,
                                  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x14661e8f6820>,
                                  'sklearn_classifier': RandomForestClassifier(bootstrap=False, max_features=3, min_samples_leaf=3,
                       min_samples_split=10, n_estimators=512, n_jobs=1,
                       random_state=42, warm_start=True)},
                              {   'balancing': Balancing(random_state=42),
                                  'classifier

In [9]:
for run_key in estimator_askl.automl_.runhistory_.data:
    print('#########')
    print(run_key)
    print(estimator_askl.automl_.runhistory_.data[run_key])

#########
RunKey(config_id=1, instance_id='{"task_id": "australian"}', seed=0, budget=0.0)
RunValue(cost=0.07995647221434724, time=5.606245994567871, status=<StatusType.SUCCESS: 1>, starttime=1651432403.6242995, endtime=1651432409.256709, additional_info={'duration': 5.383122205734253, 'num_run': 2, 'train_loss': 0.0, 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=2, instance_id='{"task_id": "australian"}', seed=0, budget=0.0)
RunValue(cost=0.1046363199755021, time=10.805355072021484, status=<StatusType.SUCCESS: 1>, starttime=1651432409.3200202, endtime=1651432420.1598368, additional_info={'duration': 10.652907133102417, 'num_run': 3, 'train_loss': 0.01231466128951087, 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=3, instance_id='{"task_id": "australian"}', seed=0, budget=0.0)
RunValue(cost=0.11166716906562166, time=8.36499309539795, status=<StatusType.SUCCESS: 1>, starttime=1651432420.305089, endtime=1651432428.7030177, additional_info={'

RunValue(cost=0.11778682943196629, time=2.01554536819458, status=<StatusType.SUCCESS: 1>, starttime=1651434987.680983, endtime=1651434989.7264, additional_info={'duration': 1.8510255813598633, 'num_run': 258, 'train_loss': 0.08937467766590036, 'configuration_origin': 'Local Search'})
#########
RunKey(config_id=258, instance_id='{"task_id": "australian"}', seed=0, budget=0.0)
RunValue(cost=0.3792378847090094, time=1.9929649829864502, status=<StatusType.SUCCESS: 1>, starttime=1651434990.183557, endtime=1651434992.2021446, additional_info={'duration': 1.8533751964569092, 'num_run': 259, 'train_loss': 0.29242841057376895, 'configuration_origin': 'Random Search'})
#########
RunKey(config_id=259, instance_id='{"task_id": "australian"}', seed=0, budget=0.0)
RunValue(cost=0.1385803820385196, time=5.784508466720581, status=<StatusType.SUCCESS: 1>, starttime=1651434992.657563, endtime=1651434998.47469, additional_info={'duration': 5.486747741699219, 'num_run': 260, 'train_loss': 0.13853370516432